#**99 - modelo solución**

#**Integrantes**

*   SANTIAGO VILLEGAS NARANJO
*   JENNY ANDREA OROZCO OSORIO
*   XIOMARA GIRALDO PÉREZ



#**Instalación de CatBoost**

In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.6 MB/s eta 0:00:00


#**Librerias**

In [ ]:
import pandas as pd
import numpy as np
import os
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
import warnings

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"xiomaragiraldoprez","key":"27128718444a1ef0ccbc265ad597de45"}'}

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.17GB/s]


#**Descompresión e inspección del archivo**

In [ ]:
!unzip udea*.zip > /dev/null

#**Cargar datasets**

In [ ]:
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")
lentr = len(train)

In [ ]:
print(f"\n Dimensiones del dataset:")
print(f"   Training: {train.shape}")
print(f"   Test: {test.shape}")


 Dimensiones del dataset:
   Training: (692500, 21)
   Test: (296786, 20)


#**limpieza de cada columna**

#**Estratos**

In [ ]:
def organizar_estratos(data, columna):
    """
    Limpia y normaliza la columna de estratos socioeconómicos.
    Convierte texto a valores numéricos (0-6).
    """
    data[columna] = (
        data[columna]
        .str.lower()
        .str.replace("sin estrato", "0", regex=False)
        .str.replace("estrato ", "", regex=False)
    )

    moda = data[columna].mode().values[0]
    data[columna] = data[columna].fillna(moda).astype(int)

    return data


#**Se convierte el si o no en 1 o 0**

In [ ]:
def organizar_columna_binaria(data, columna, valores):
    """
    Convierte columnas binarias (Sí/No) a valores numéricos (1/0).
    Rellena NaN con la moda por estrato.
    """
    modas = data.groupby('F_ESTRATOVIVIENDA')[columna].transform(
        lambda x: x.mode().iloc[0] if not x.mode().empty else valores[0]
    )
    data[columna] = data[columna].fillna(modas)

    data.loc[data[columna] == valores[0], columna] = 1
    data.loc[data[columna] == valores[1], columna] = 0

    return data

#**Valor de matrícula en una sola escala**

In [ ]:
def organizar_valor_matricula(data, columna):
    """
    Convierte el valor de matrícula a escala ordinal numérica (0-7).
    """
    mapeo = {
        'Más de 7 millones': 7,
        'Entre 5.5 millones y menos de 7 millones': 6,
        'Entre 4 millones y menos de 5.5 millones': 5,
        'Entre 2.5 millones y menos de 4 millones': 4,
        'Entre 1 millón y menos de 2.5 millones': 3,
        'Entre 500 mil y menos de 1 millón': 2,
        'Menos de 500 mil': 1,
        'No pagó matrícula': 0
    }

    modas = data.groupby('F_ESTRATOVIVIENDA')[columna].transform(
        lambda x: x.mode().iloc[0] if not x.mode().empty else 'No pagó matrícula'
    )
    data[columna] = data[columna].fillna(modas)
    data[columna] = data[columna].map(mapeo)

    return data


#**Horas trabajadas del estudiante**

In [ ]:
def organizar_horas_trabajadas_estudiante(data, columna):
    """
    Convierte las horas de trabajo de texto a valores numéricos ordinales.
    Rellena NaN con la moda por estrato.
    """
    mapeo = {
        "0": 0,
        "menos de 10 horas": 1,
        "entre 11 y 20 horas": 2,
        "entre 21 y 30 horas": 3,
        "más de 30 horas": 4
    }

    # Rellenar NaN con moda por estrato
    modas = data.groupby('F_ESTRATOVIVIENDA')[columna].transform(
        lambda x: x.mode().iloc[0] if not x.mode().empty else "0"
    )
    data[columna] = data[columna].fillna(modas)

    # Normalizar y mapear
    data[columna] = data[columna].astype(str).str.strip().str.lower()
    data[columna] = data[columna].map(mapeo)

    return data

#**Nivel educativo de los padres**

In [ ]:
def organizar_educacion_padres(data, columna):
    """
    Convierte el nivel educativo de los padres a escala ordinal (0-11).
    """
    mapeo = {
        'No sabe': 0,
        'Ninguno': 1,
        'No Aplica': 2,
        'Primaria incompleta': 3,
        'Primaria completa': 4,
        'Secundaria (Bachillerato) incompleta': 5,
        'Secundaria (Bachillerato) completa': 6,
        'Técnica o tecnológica incompleta': 7,
        'Técnica o tecnológica completa': 8,
        'Educación profesional incompleta': 9,
        'Educación profesional completa': 10,
        'Postgrado': 11,
    }

    modas = data.groupby('F_ESTRATOVIVIENDA')[columna].transform(
        lambda x: x.mode().iloc[0] if not x.mode().empty else 'No sabe'
    )
    data[columna] = data[columna].fillna(modas)
    data[columna] = data[columna].map(mapeo)

    return data


#**Los periodos según ciclos académicos**

In [ ]:
def organizar_periodos(data, columna):
    """
    Agrupa periodos académicos en categorías temporales más amplias.
    """
    mapeo = {
        20183: 0, 20184: 0,  # 2018
        20194: 1, 20195: 1, 20196: 1,  # 2019
        20202: 2, 20203: 2,  # 2020
        20212: 3, 20213: 3   # 2021
    }

    data[columna] = data[columna].map(mapeo)

    return data

#**Columna que suma cuántas comodidades tiene el hogar**

In [ ]:
def crear_columna_comodidades(data, columnas=None):
    """
    Crea un índice de comodidades sumando la disponibilidad de bienes.
    """
    if columnas is None:
        columnas = [
            "F_TIENELAVADORA",
            "F_TIENEAUTOMOVIL",
            "F_TIENEINTERNET",
            "F_TIENECOMPUTADOR",
            "E_PAGOMATRICULAPROPIO"
        ]

    data['Comodidades'] = data[columnas].sum(axis=1)

    return data

#**Limpieza completa del dataset**

In [ ]:
def clean_data(data):
    """
    Función principal de limpieza que aplica todas las transformaciones.
    """
    print("\n Iniciando limpieza de datos...")

    # Eliminar columnas innecesarias
    columnas_eliminar = [
        "ID",
        "INDICADOR_4",  # Alta correlación con coef_2
        "F_TIENEINTERNET.1",  # Duplicado
        "E_PRIVADO_LIBERTAD"  # Poca variabilidad
    ]

    data = data.drop(columns=[col for col in columnas_eliminar if col in data.columns])

    # Aplicar transformaciones
    data = organizar_estratos(data, "F_ESTRATOVIVIENDA")
    data = organizar_periodos(data, "PERIODO_ACADEMICO")
    data = organizar_horas_trabajadas_estudiante(data, "E_HORASSEMANATRABAJA")

    # Variables binarias
    binarias = {
        "F_TIENEINTERNET": ["Si", "No"],
        "F_TIENECOMPUTADOR": ["Si", "No"],
        "F_TIENELAVADORA": ["Si", "No"],
        "F_TIENEAUTOMOVIL": ["Si", "No"],
        "E_PAGOMATRICULAPROPIO": ["Si", "No"]
    }

    for col, valores in binarias.items():
        data = organizar_columna_binaria(data, col, valores)

    # Variables ordinales
    data = organizar_valor_matricula(data, "E_VALORMATRICULAUNIVERSIDAD")
    data = organizar_educacion_padres(data, "F_EDUCACIONPADRE")
    data = organizar_educacion_padres(data, "F_EDUCACIONMADRE")

    # Crear índice de comodidades
    data = crear_columna_comodidades(data)

    # Eliminar columnas redundantes después de crear el índice
    data = data.drop(columns=["F_TIENEAUTOMOVIL"])

    print(" Limpieza completada")
    print(f"   Columnas finales: {data.shape[1]}")
    print(f"   Registros: {data.shape[0]}")

    return data

#**train y test se limpian igual**

In [ ]:
print("PREPARACIÓN DE DATOS")

# Combinar train y test para limpieza consistente
source_cols = [col for col in train.columns if col != "RENDIMIENTO_GLOBAL"]
all_data = pd.concat([train[source_cols], test[source_cols]], ignore_index=True)

# Limpiar datos
all_data = clean_data(all_data)

# Separar train y test
lentr = len(train)
Xtr = all_data.iloc[:lentr].values
ytr = train["RENDIMIENTO_GLOBAL"].values
Xts = all_data.iloc[lentr:].values

print(f"\n Dimensiones finales:")
print(f"   X_train: {Xtr.shape}")
print(f"   y_train: {ytr.shape}")
print(f"   X_test: {Xts.shape}")

# Mostrar ejemplo de datos procesados
print(f"\n Muestra de datos procesados:")
print(all_data.head())


PREPARACIÓN DE DATOS

 Iniciando limpieza de datos...
 Limpieza completada
   Columnas finales: 16
   Registros: 989286

 Dimensiones finales:
   X_train: (692500, 16)
   y_train: (692500,)
   X_test: (296786, 16)

 Muestra de datos procesados:
   PERIODO_ACADEMICO            E_PRGM_ACADEMICO E_PRGM_DEPARTAMENTO  \
0                  3                  ENFERMERIA              BOGOTÁ   
1                  3                     DERECHO           ATLANTICO   
2                  2       MERCADEO Y PUBLICIDAD              BOGOTÁ   
3                  1  ADMINISTRACION DE EMPRESAS           SANTANDER   
4                  3                  PSICOLOGIA           ANTIOQUIA   

   E_VALORMATRICULAUNIVERSIDAD  E_HORASSEMANATRABAJA  F_ESTRATOVIVIENDA  \
0                            6                     1                  3   
1                            4                     0                  3   
2                            4                     4                  3   
3                    

#**CatBoost**

In [ ]:
print("ENTRENAMIENTO DEL MODELO")

# División train-validation
Xtrain, Xval, ytrain, yval = train_test_split(
    Xtr, ytr,
    test_size=0.2,
    random_state=42,
    stratify=ytr
)

print(f"\n División de datos:")
print(f"   Entrenamiento: {Xtrain.shape[0]:,} registros")
print(f"   Validación: {Xval.shape[0]:,} registros")

# Identificar columnas categóricas
cat_features = [0, 1, 2]  # PERIODO, ESTU_PRGM_ACADEMICO, ESTU_PRGM_DEPARTAMENTO

# Configurar modelo CatBoost
model = CatBoostClassifier(
    iterations=1200,
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=5,
    bootstrap_type='Bernoulli',
    subsample=0.8,
    auto_class_weights='Balanced',
    thread_count=-1,
    random_seed=42,
    verbose=100,
    task_type='CPU',
    eval_metric='Accuracy',
    cat_features=cat_features,
    early_stopping_rounds=50
)

print(f"\n Iniciando entrenamiento...")
print(f"   Configuración del modelo:")
print(f"   - Iteraciones máximas: 1200")
print(f"   - Learning rate: 0.05")
print(f"   - Profundidad: 8")
print(f"   - Early stopping: 50 rondas")

# Entrenar modelo
model.fit(
    Xtrain, ytrain,
    eval_set=(Xval, yval),
    use_best_model=True,
    plot=False
)



ENTRENAMIENTO DEL MODELO

 División de datos:
   Entrenamiento: 554,000 registros
   Validación: 138,500 registros

 Iniciando entrenamiento...
   Configuración del modelo:
   - Iteraciones máximas: 1200
   - Learning rate: 0.05
   - Profundidad: 8
   - Early stopping: 50 rondas
0:	learn: 0.3988961	test: 0.3995149	best: 0.3995149 (0)	total: 5.51s	remaining: 1h 50m 12s
100:	learn: 0.4350209	test: 0.4317247	best: 0.4318238 (99)	total: 6m 19s	remaining: 1h 8m 53s
200:	learn: 0.4424564	test: 0.4363618	best: 0.4364132 (198)	total: 12m 29s	remaining: 1h 2m 4s
300:	learn: 0.4473659	test: 0.4386823	best: 0.4386823 (300)	total: 18m 41s	remaining: 55m 48s
400:	learn: 0.4511421	test: 0.4400967	best: 0.4401473 (396)	total: 24m 57s	remaining: 49m 44s
500:	learn: 0.4542297	test: 0.4410955	best: 0.4411398 (497)	total: 31m 16s	remaining: 43m 38s
600:	learn: 0.4565831	test: 0.4417163	best: 0.4418026 (589)	total: 37m 42s	remaining: 37m 35s
700:	learn: 0.4588771	test: 0.4420663	best: 0.4422320 (686)	tot

#**EVALUACIÓN DEL MODELO**

In [ ]:

print("EVALUACIÓN DEL MODELO")

# Predicciones
ytrain_pred = model.predict(Xtrain)
yval_pred = model.predict(Xval)

# Métricas
train_acc = accuracy_score(ytrain, ytrain_pred)
val_acc = accuracy_score(yval, yval_pred)

print(f"\n Accuracy:")
print(f"   Entrenamiento: {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"   Validación: {val_acc:.4f} ({val_acc*100:.2f}%)")
print(f"   Diferencia: {(train_acc - val_acc)*100:.2f}%")

# Reporte de clasificación
print(f"\n Reporte de clasificación (Validación):")
print(classification_report(yval, yval_pred))

# Importancia de características
feature_importance = model.get_feature_importance()
feature_names = all_data.columns.tolist()

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

print(f"\n Top 10 características más importantes:")
print(importance_df.head(10).to_string(index=False))

EVALUACIÓN DEL MODELO

 Accuracy:
   Entrenamiento: 0.4568 (45.68%)
   Validación: 0.4434 (44.34%)
   Diferencia: 1.34%

 Reporte de clasificación (Validación):
              precision    recall  f1-score   support

        alto       0.58      0.62      0.60     35124
        bajo       0.47      0.58      0.52     34597
  medio-alto       0.33      0.30      0.32     34324
  medio-bajo       0.34      0.27      0.30     34455

    accuracy                           0.44    138500
   macro avg       0.43      0.44      0.43    138500
weighted avg       0.43      0.44      0.43    138500


 Top 10 características más importantes:
                    Feature  Importance
           E_PRGM_ACADEMICO   32.993455
E_VALORMATRICULAUNIVERSIDAD   18.560747
        E_PRGM_DEPARTAMENTO   16.655349
           F_EDUCACIONMADRE    5.536509
       E_HORASSEMANATRABAJA    4.578175
          PERIODO_ACADEMICO    4.518904
           F_EDUCACIONPADRE    4.302194
      E_PAGOMATRICULAPROPIO    2.738690
  

#**PREDICCIÓN Y ENVÍO**

In [ ]:
#print("\n" + "="*70)
print("GENERACIÓN DE PREDICCIONES")
#print("="*70)

# Predicciones en test set
yts_pred = model.predict(Xts)

# Asegurar que yts_pred sea un array 1D
if yts_pred.ndim > 1:
    yts_pred = yts_pred.flatten()

print(f"\n Verificando dimensiones:")
print(f"   yts_pred shape: {yts_pred.shape}")
print(f"   df_test shape: {test.shape}")

# Crear archivo de submission
submission = pd.DataFrame({
    'ID': test['ID'].values,
    'RENDIMIENTO_GLOBAL': yts_pred
})

submission.to_csv('predict.csv', index=False)

print(f"\n Archivo de predicciones generado: predict.csv")
print(f"   Total de predicciones: {len(submission):,}")
print(f"\n Distribución de predicciones:")
print(submission['RENDIMIENTO_GLOBAL'].value_counts().sort_index())
print(f"\n{submission.head(10)}")

#print("\n" + "="*70)
print("PROCESO COMPLETADO ")
#print("="*70)

GENERACIÓN DE PREDICCIONES

 Verificando dimensiones:
   yts_pred shape: (296786,)
   df_test shape: (296786, 20)

 Archivo de predicciones generado: predict.csv
   Total de predicciones: 296,786

 Distribución de predicciones:
RENDIMIENTO_GLOBAL
alto          80129
bajo          91777
medio-alto    66684
medio-bajo    58196
Name: count, dtype: int64

       ID RENDIMIENTO_GLOBAL
0  550236               bajo
1   98545         medio-alto
2  499179               alto
3  782980               bajo
4  785185               bajo
5   58495               bajo
6  705444               alto
7  557548               alto
8  519909               bajo
9  832058               alto
PROCESO COMPLETADO 


#**Subir a kaggle**

In [ ]:
!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia \
 -f predict.csv \
 -m "Segundo envío"

100% 4.08M/4.08M [00:02<00:00, 1.68MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia